In [1]:
import pandas as pd
genuine_accounts_users = pd.read_csv('cresci-2017.csv/datasets_full.csv/genuine_accounts.csv/users.csv')
genuine_accounts_tweets = pd.read_csv('cresci-2017.csv/datasets_full.csv/genuine_accounts.csv/tweets.csv')
social_spambots_2_users = pd.read_csv('cresci-2017.csv/datasets_full.csv/social_spambots_2.csv/users.csv')
social_spambots_2_tweets = pd.read_csv('cresci-2017.csv/datasets_full.csv/social_spambots_2.csv/tweets.csv')
social_spambots_3_users = pd.read_csv('cresci-2017.csv/datasets_full.csv/social_spambots_2.csv/users.csv')
social_spambots_3_tweets = pd.read_csv('cresci-2017.csv/datasets_full.csv/social_spambots_2.csv/tweets.csv')
traditional_spambots_1_users = pd.read_csv('cresci-2017.csv/datasets_full.csv/traditional_spambots_1.csv/users.csv', low_memory=False)
traditional_spambots_1_tweets = pd.read_csv('cresci-2017.csv/datasets_full.csv/traditional_spambots_1.csv/tweets.csv', low_memory=False)
fake_followers_users=pd.read_csv('cresci-2017.csv/datasets_full.csv/fake_followers.csv/users.csv', low_memory=False)
fake_followers_tweets=pd.read_csv('cresci-2017.csv/datasets_full.csv/fake_followers.csv/tweets.csv', low_memory=False)
russian_troll_users = pd.read_csv('russian-troll-tweets/users.csv', low_memory=False)
russian_troll_tweets = pd.read_csv('russian-troll-tweets/tweets.csv', low_memory=False)

/Users/xinchunli/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/xinchunli/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [191]:
a1 = genuine_accounts_users
a2 = genuine_accounts_tweets
b1=fake_followers_users
b2= fake_followers_tweets

In [192]:
import random
a1.rename(columns={'id': 'user_id'}, inplace=True)
a_merged1 = pd.merge(a1, a2, on = 'user_id', how = 'outer').sample(n=1000, random_state=1)
a_merged2 = pd.merge(a1, a2, on = 'user_id', how = 'outer').drop(index=a_merged1.index).sample(n=1000, random_state=0)
b1.rename(columns={'id': 'user_id'}, inplace=True)
b_merged1 = pd.merge(b1, b2, on = 'user_id', how = 'outer').sample(n=1000, random_state=1)
b_merged2 = pd.merge(b1, b2, on = 'user_id', how = 'outer').drop(index=b_merged1.index).sample(n=100, random_state=0)

In [193]:
a_merged1['label']=int(0)
a_merged2['label']=int(0)
b_merged1['label']=int(1)
b_merged2['label']=int(1)

In [194]:
a_merged1=a_merged1[['user_id', 'statuses_count', 'followers_count', 'friends_count', 'utc_offset', 
                   'profile_use_background_image', 'favourites_count', 'listed_count','retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions', 'label']]

In [195]:
a_merged2=a_merged2[['user_id', 'statuses_count', 'followers_count', 'friends_count', 'utc_offset', 
                   'profile_use_background_image', 'favourites_count', 'listed_count','retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions', 'label']]

In [196]:
b_merged1=b_merged1[['user_id', 'statuses_count', 'followers_count', 'friends_count', 'utc_offset', 
                   'profile_use_background_image', 'favourites_count', 'listed_count', 'retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions', 'label']]

In [197]:
b_merged2=b_merged2[['user_id', 'statuses_count', 'followers_count', 'friends_count', 'utc_offset', 
                   'profile_use_background_image', 'favourites_count', 'listed_count','retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions', 'label']]

In [198]:
ab_agg = pd.concat([a_merged1, b_merged1])
a1b1_agg=pd.concat([a_merged2, b_merged2])

In [243]:
b_merged3 = pd.merge(b1, b2, on = 'user_id', how = 'outer').drop(index=b_merged1.index).drop(index=b_merged2.index).sample(n=10, random_state=0)
b_merged3['label']=int(1)
b_merged3=b_merged3[['user_id', 'statuses_count', 'followers_count', 'friends_count', 'utc_offset', 'profile_use_background_image', 'favourites_count', 'listed_count','retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions', 'label']]
ab_agg = pd.concat([ab_agg, b_merged3])



In [244]:
ab_agg.shape

(240000, 15)

In [245]:
for column in ab_agg.columns:
    ab_agg[column]=ab_agg[column].fillna(ab_agg[ab_agg[column].isnull()==False][column].median())

In [246]:
for column in a1b1_agg.columns:
    a1b1_agg[column]=a1b1_agg[column].fillna(a1b1_agg[a1b1_agg[column].isnull()==False][column].median())

In [247]:
from sklearn.utils import shuffle
ab_agg=shuffle(ab_agg)
a1b1_agg=shuffle(a1b1_agg)

In [248]:
X_train = ab_agg.iloc[:,1:14]
y_train=ab_agg['label']
X_test = a1b1_agg.iloc[:,1:14]
y_test=a1b1_agg['label']

In [249]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

# Baseline Models:

## Logistic Regression Classifier

In [250]:
# build a logistic regression
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(penalty="l1",random_state=0).fit(X_train_transformed, y_train)
import numpy as np
predicted = lr_model.predict(X_test_transformed)
print(np.mean(predicted == y_test))
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))

0.8535
              precision    recall  f1-score   support

           0       0.97      0.73      0.83     70000
           1       0.78      0.98      0.87     70000

   micro avg       0.85      0.85      0.85    140000
   macro avg       0.88      0.85      0.85    140000
weighted avg       0.88      0.85      0.85    140000



## Random Forest Classifier

In [251]:
# build a Random Forest Classifier
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(max_depth=5, criterion='entropy',
                                   random_state=0).fit(X_train_transformed, y_train)
import numpy as np
predicted = rf_model.predict(X_test_transformed)
print(np.mean(predicted == y_test))
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))

0.9872071428571428
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     70000
           1       0.98      1.00      0.99     70000

   micro avg       0.99      0.99      0.99    140000
   macro avg       0.99      0.99      0.99    140000
weighted avg       0.99      0.99      0.99    140000



## GDBoosting Classifier

In [252]:
# build a Random Forest Classifier
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import GradientBoostingClassifier
gdb_model = GradientBoostingClassifier(learning_rate=1.0, max_depth=3, random_state=0).fit(X_train_transformed, y_train)
import numpy as np
predicted = gdb_model.predict(X_test_transformed)
print(np.mean(predicted == y_test))
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))

0.9987357142857143
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     70000
           1       1.00      1.00      1.00     70000

   micro avg       1.00      1.00      1.00    140000
   macro avg       1.00      1.00      1.00    140000
weighted avg       1.00      1.00      1.00    140000



## SVM Classifier

In [253]:
# build a Random Forest Classifier
import warnings
warnings.filterwarnings('ignore')
from sklearn.svm import SVC
svm_model = SVC(gamma='auto', max_iter=100).fit(X_train_transformed, y_train)
import numpy as np
predicted = svm_model.predict(X_test_transformed)
print(np.mean(predicted == y_test))
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))

0.7916642857142857
              precision    recall  f1-score   support

           0       0.93      0.63      0.75     70000
           1       0.72      0.95      0.82     70000

   micro avg       0.79      0.79      0.79    140000
   macro avg       0.82      0.79      0.79    140000
weighted avg       0.82      0.79      0.79    140000

